In [ ]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/한글train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/한글test.csv')
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/sample.csv', encoding='cp949')

In [ ]:
display(train.head(3))

,접수년월,질병경중등급코드,KCD등급코드,질병구분코드,발생지역구분코드,치료행위코드,의료기관구분코드,요양병원여부,고객나이구분코드,부담보5년경과여부,부실모집설계사계약여부,중도부가계약여부,건강인우대계약가입여부,보험사기이력고객여부,보험료구간코드,가입금액구간코드,통원횟수,청구보험금,입원일수,청구일계약일간기간구분코드,청구일부활일간기간구분코드,병원별평균입원청구보험금,병원별평균통원청구보험금,병원별평균수술청구보험금,병원별평균진단청구보험금,질병별평균입원청구보험금,질병별평균통원청구보험금,질병별평균수술청구보험금,질병별평균진단청구보험금,입원청구건수,수술청구건수,통원청구건수,target
0,201901,1,3,1,1,12,2,0,3,0,0,0,0,0,1,4,0.8571,0.4240,0.3452,6,0,1.6021,0.1303,0.0000,0.0,1.2612,1.2002,1.1701,0.9793,10.0,0.0,2.0,1
1,201901,3,8,22,4,2,1,1,4,0,0,0,0,0,1,4,0.0000,0.8602,0.0000,6,0,0.0000,0.0000,0.5770,0.0,0.0023,0.0000,0.1508,0.0000,0.0,1.0,0.0,0
2,201901,3,8,22,1,2,1,1,4,0,0,0,0,1,1,6,0.0000,0.4301,0.0000,6,0,0.0000,0.0000,0.2885,0.0,0.0023,0.0000,0.1508,0.0000,0.0,1.0,0.0,1


# **Baseline Model Select**
> 11월 데이터의 target을 예측하여 성능 측정


### 1) 전체 데이터를 다 쓰는 경우

In [ ]:
local_train_all = train.query('접수년월!=201911')
local_test = train.query('접수년월==201911')

X = local_train_all.drop(['target'], axis=1)
y = local_train_all['target']

X_test = local_test.drop(['target'], axis=1)
y_test = local_test['target']

In [ ]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(342968, 32) (342968,) (34960, 32) (34960,)


#### A. 선형모델
- Ridge
- SVM
- logistic(softmax)
- KNN
- Naive Bayes
- Decision Tree

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ridge
ridge = RidgeClassifier(random_state = 1993)
ridge.fit(X, y)
ridge_pred = ridge.predict(X_test)
target_names = ['자동지급', '심사', '조사']
print('< Ridge Classifier >')
print(classification_report(y_test, ridge_pred, target_names = target_names))
print(accuracy_score(y_test, ridge_pred))
print('-'* 50)

# logistic
logit = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logit.fit(X, y)
logit_pred = logit.predict(X_test)
print('< Logistic >')
print(classification_report(y_test, logit_pred, target_names = target_names))
print(accuracy_score(y_test, logit_pred))
print('-'* 50)

# svc
svc = SVC()
svc.fit(X, y)
svc_pred = svc.predict(X_test)
print('< SVC >')
print(classification_report(y_test, svc_pred, target_names = target_names))
print(accuracy_score(y_test, svc_pred))
print('-'* 50)

# naive bayes
nb = GaussianNB()
nb.fit(X, y)
nb_pred = nb.predict(X_test)
print('< Naive Bayes >')
print(classification_report(y_test, nb_pred, target_names = target_names))
print(accuracy_score(y_test, nb_pred))
print('-'* 50)

# knn
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X, y)
knn_pred = knn.predict(X_test)
print('< KNN >')
print(classification_report(y_test, knn_pred, target_names = target_names))
print(accuracy_score(y_test, knn_pred))

# Decition tree
dt = DecisionTreeClassifier()
dt.fit(X, y)
dt_pred = dt.predict(X_test)
print('< DT >')
print(classification_report(y_test,dt_pred, target_names = target_names))
print(accuracy_score(y_test, dt_pred))

#### B. 앙상블 계열
> 되도록 oof로 쌓을 것

- lgbm
- xgb
- catboost
- rf

In [ ]:
!pip install catboost
from sklearn.model_selection import KFold, StratifiedKFold  
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss

random_seed = 2020

# CatBoost 모델링
def catboost_modeling(x_train, y_train, x_test, grow_policy, depth, learning_rate, l2_leaf_reg, random_seed, n):
  
  train_pred = np.zeros((len(x_train), 3))
  test_pred = np.zeros((len(x_test), 3))
  
  for i in range(n):
    # kf = KFold(n_splits=10, random_state=random_seed+i)
    skf = StratifiedKFold(n_splits=5, random_state=74, shuffle=True)
    
    for idx, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
      # train_X, train_y = x_train.iloc[train_index], y_train[train_index]
      # valid_X, valid_y = x_train.iloc[valid_index], y_train[valid_index]

      train_X, valid_X = x_train.iloc[train_index], x_train.iloc[valid_index]
      train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]
      print("\nFold", idx)
      
      # catBoost(grow_policy='Depthwise')
      model = CatBoostClassifier( loss_function = 'MultiClass', 
                                #  eval_metric = 'AUC',              
                                 iterations = 25000,               
                                 metric_period = 25000,            
                                 early_stopping_rounds = 1000,     
                                 task_type = 'GPU',                
                                 grow_policy = grow_policy,        # 트리 노드 생성 방식
                                                                   # 1) Depthwise(지정한 depth에 이를 때까지 level 순으로 노드 분할)
                                                                   # 2) Lossguide(loss 변화가 큰 순으로 노드 분할)
                                 depth = depth,                  
                                 learning_rate = learning_rate,    
                                 l2_leaf_reg = l2_leaf_reg,        
                                 random_seed = random_seed+i,      
                                 )
      # 모델 학습
      model.fit(train_X, train_y, eval_set=(valid_X, valid_y))
    
      # 모델 적용
      pred_val = model.predict_proba(valid_X)
      print( " Log loss = ", log_loss(valid_y, pred_val))

      # train_pred[valid_index] = pred

      test_pred += model.predict_proba(x_test)

  test_pred /= (5*n)

  return test_pred

     |████████████████████████████████| 66.1MB 49kB/s 


In [ ]:
# %%time
# catboost_modeling(real_X, real_y, real_X_test, 'Lossguide', 16, 0.01213, 5.027, 2022, 2) # 2번 반복

In [ ]:
def lgbm_classifier(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2284, num_rounds=118903):

    param = {}

#     param['boosting'] = 'dart'
    
    param['objective'] = 'multiclass'

    param['learning_rate'] = 0.05

    param['max_depth'] = 18

#     param['silent'] = 1

    param['metric'] = 'multi_logloss'
    
    param['is_training_metric'] = True
    
    param['min_child_weight'] = 1

    param['bagging_fraction'] = 0.85
    
    param['num_leaves'] = 255

    param['feature_fraction'] = 0.8

    param['bagging_freq'] = 3
    
    param['seed'] = 2284

    param['num_class'] = 3
    
    param['min_split_gain'] = 0.01

    param['nthread'] = 64 

    param['min_data_in_leaf'] = 4

    
#     param['categorical_feature'] = 'name:fiberID'
    
#     param['lambda_l2'] = 0.2

#     param['lambda_l1'] = 0.2
    
#     lambda_l1 

    num_rounds = num_rounds
    plst = list(param.items())
    train_ds = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        test_ds = lgb.Dataset(test_X, label=test_y)
        watchlist = [test_ds]
        model = lgb.train(param, train_ds, num_rounds,test_ds, early_stopping_rounds=180)

    else:
        model = lgb.train(plst, test_ds, 10000)
    return model

In [ ]:
import lightgbm as lgb

def lgbm_modeling(x_train, y_train, x_test, n):

  test_pred = np.zeros((len(x_test), 3))
  
  for i in range(n):
    skf = StratifiedKFold(n_splits=5, random_state=74, shuffle=True)

    for idx, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):

      train_X, valid_X = x_train.iloc[train_index], x_train.iloc[valid_index]
      train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]
      print("\nFold", idx)

      model = lgbm_classifier(train_X, train_y, valid_X, valid_y)

      pred = model.predict(x_test, num_iteration=model.best_iteration)
    
      test_pred += pred / (5*n)
  
    y_pred = [np.argmax(line) for line in test_pred]
    
    # submission
    sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/sample.csv', encoding='cp949')
    sub['target'] = y_pred
    sub.to_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/sub/lgbm2.csv', index=False)

  # return test_pred, y_pred

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/한글train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/한글test.csv')

real_X = train.drop(['target'], axis=1)
real_y = train['target']
real_X_test = test

In [ ]:
print(real_X.shape, real_y.shape, real_X_test.shape)

(377928, 32) (377928,) (22072, 32)


In [ ]:
print(train.shape, test.shape)

(377928, 33) (22072, 32)


In [ ]:
from sklearn.model_selection import train_test_split
tr_x, val_x, tr_y, val_y = train_test_split(real_X, real_y, test_size=0.2, random_state=20)

In [ ]:
print(tr_x.shape, tr_y.shape, val_x.shape, val_y.shape)

(302342, 32) (302342,) (75586, 32) (75586,)


In [ ]:
# 단일
import lightgbm as lgb
model = lgbm_classifier(tr_x, tr_y, val_x, val_y)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[11288]	valid_0's multi_logloss: 0.0886962
[11289]	valid_0's multi_logloss: 0.0886922
[11290]	valid_0's multi_logloss: 0.0886864
[11291]	valid_0's multi_logloss: 0.0886831
[11292]	valid_0's multi_logloss: 0.088679
[11293]	valid_0's multi_logloss: 0.0886764
[11294]	valid_0's multi_logloss: 0.0886736
[11295]	valid_0's multi_logloss: 0.0886716
[11296]	valid_0's multi_logloss: 0.0886684
[11297]	valid_0's multi_logloss: 0.088665
[11298]	valid_0's multi_logloss: 0.0886615
[11299]	valid_0's multi_logloss: 0.0886584
[11300]	valid_0's multi_logloss: 0.0886563
[11301]	valid_0's multi_logloss: 0.0886509
[11302]	valid_0's multi_logloss: 0.0886479
[11303]	valid_0's multi_logloss: 0.0886437
[11304]	valid_0's multi_logloss: 0.0886398
[11305]	valid_0's multi_logloss: 0.0886361
[11306]	valid_0's multi_logloss: 0.0886316
[11307]	valid_0's multi_logloss: 0.0886283
[11308]	valid_0's multi_logloss: 0.0886246
[11309]	valid_0's multi_logloss: 0.0886209
[11310]	valid_0's mu

In [ ]:
y_test =  model.predict(test, num_iteration=model.best_iteration)

In [ ]:
y_pred_1 = [np.argmax(line) for line in y_test]
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/data/sample.csv', encoding='cp949')
sub['target'] = y_pred_1
sub.to_csv('/content/drive/My Drive/Colab Notebooks/미래에셋/sub/lgbm_on.csv', index=False)

In [ ]:
# # baysian 
# !pip install bayesian-optimization
# from bayes_opt import BayesianOptimization
# from functools import partial

# def lgbm_bay(train_X, train_y, test_X, test_y, num_rounds, learning_rate, max_depth, num_leaves, bagging_fraction, feature_fraction, reg_alpha, reg_lambda):

#     param = {}

# #     param['boosting'] = 'dart'
    
#     param['objective'] = 'multiclass'

#     param['learning_rate'] = learning_rate

#     param['max_depth'] = int(max_depth)

#     param['silent'] = 0

#     param['metric'] = 'multi_logloss'
    
#     param['is_training_metric'] = True
    
#     param['min_child_weight'] = 1

#     param['bagging_fraction'] = bagging_fraction
    
#     param['num_leaves'] = int(num_leaves)

#     param['feature_fraction'] = feature_fraction

#     param['bagging_freq'] = 6
    
#     param['seed'] = 2284

#     param['num_class'] = 3
    
#     param['min_split_gain'] = 0.01
    
# #     param['categorical_feature'] = 'name:fiberID'
    
#     param['lambda_l2'] = reg_lambda

#     param['lambda_l1'] = reg_alpha

#     num_rounds = int(num_rounds)

#     plst = list(param.items())
#     train_ds = lgb.Dataset(train_X, label=train_y)

#     if test_y is not None:
#         test_ds = lgb.Dataset(test_X, label=test_y)
#         watchlist = [test_ds]
#         model = lgb.train(param, train_ds, int(num_rounds), test_ds, early_stopping_rounds=180)

#     else:
#         model = lgb.train(plst, test_ds, 10000)
#     return model

In [ ]:
# func_fixed = partial(lgbm_bay, train_X = tr_x, train_y = tr_y, test_X = val_x, test_y = val_y)

# lgb_bay = BayesianOptimization(func_fixed, 
#                                {  'num_rounds' : (1000, 100000),
#                                   'learning_rate' : (0.0001, 0.1), 
#                                   'max_depth' : (16, 1024),
#                                   'num_leaves' : (16, 1024),
#                                   'bagging_fraction' : (0,1),
#                                   'feature_fraction' : (0,1),
#                                   'reg_alpha' : (0, 10),
#                                   'reg_lambda' : (0, 50),
#                                }, 
#                                random_state = 2020)


# # 처음 5회 랜덤값으로 score 계산 후 30회 최적화
# lgb_bay.maximize(init_points = 5, n_iter=40)

# params = lgb_bay.max['params']
# params

In [ ]:
# 앙상블
%%time
lgbm_modeling(real_X, real_y, real_X_test, 1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[1826]	valid_0's multi_logloss: 0.1014
[1827]	valid_0's multi_logloss: 0.101371
[1828]	valid_0's multi_logloss: 0.101339
[1829]	valid_0's multi_logloss: 0.101312
[1830]	valid_0's multi_logloss: 0.101287
[1831]	valid_0's multi_logloss: 0.101269
[1832]	valid_0's multi_logloss: 0.101249
[1833]	valid_0's multi_logloss: 0.101205
[1834]	valid_0's multi_logloss: 0.101162
[1835]	valid_0's multi_logloss: 0.10112
[1836]	valid_0's multi_logloss: 0.101087
[1837]	valid_0's multi_logloss: 0.10105
[1838]	valid_0's multi_logloss: 0.101018
[1839]	valid_0's multi_logloss: 0.100969
[1840]	valid_0's multi_logloss: 0.100948
[1841]	valid_0's multi_logloss: 0.100922
[1842]	valid_0's multi_logloss: 0.1009
[1843]	valid_0's multi_logloss: 0.100869
[1844]	valid_0's multi_logloss: 0.100844
[1845]	valid_0's multi_logloss: 0.100812
[1846]	valid_0's multi_logloss: 0.100784
[1847]	valid_0's multi_logloss: 0.100754
[1848]	valid_0's multi_logloss: 0.100717
[1849]	valid_0's multi_logl